In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Example usage

To use `pyinverse` in a project:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyinverse.forward_model import Linear
from pyinverse.loss import LeastSquares, Tikhonov, Bayesian
from pyinverse.solver import LSTSQ


In [111]:
# Simple forward model
x = np.array([1, 2])  # True state
K = np.array(
    [
        [1, 1],
        [0, 2],
    ]
)  # Forward matrix


In [ ]:
forward_model = Linear(K)
y = forward_model(x)  # Measurement
# loss = LeastSquares(y, K)
alpha = 1.0
loss = Tikhonov(y, K, alpha)
solver = LSTSQ(loss)  

In [103]:
for alpha in [1, 2, 3]:
    loss.alpha = alpha
    x_est = solver()[0]
    print(x_est)

[0.63636364 1.72727273]
[0.5 1.5]
[0.41935484 1.32258065]


In [104]:
x_est, res, rank, s = solver()

In [105]:
# Compute loss for the measurements
loss(x_est)


9.193548387096774

In [106]:
K @ x_est

array([1.74193548, 2.64516129])

In [107]:
x_est

array([0.41935484, 1.32258065])

In [113]:
x_prior = np.array([0, 1])
cov_prior = np.array([1,2])
loss = Bayesian(y=y, K=K, x_prior=x_prior, cov_prior=cov_prior)

In [114]:
loss(x_est)

4.4687825182101975